In [52]:
import pandas as pd 
import numpy as np
from scipy.stats import skew, kurtosis

In [53]:
# url = "https://raw.githubusercontent.com/WalePhenomenon/climate_change/master/fuel_ferc1.csv"
# fuel_data = pd.read_csv(url, error_bad_lines=False)
fuel_data = pd.read_csv("fuel_ferc1.csv")
fuel_data.head()

,record_id,utility_id_ferc1,report_year,plant_name_ferc1,fuel_type_code_pudl,fuel_unit,fuel_qty_burned,fuel_mmbtu_per_unit,fuel_cost_per_unit_burned,fuel_cost_per_unit_delivered,fuel_cost_per_mmbtu
0,f1_fuel_1994_12_1_0_7,1,1994,rockport,coal,ton,5377489.0,16.590,18.59,18.53,1.121
1,f1_fuel_1994_12_1_0_10,1,1994,rockport total plant,coal,ton,10486945.0,16.592,18.58,18.53,1.120
2,f1_fuel_1994_12_2_0_1,2,1994,gorgas,coal,ton,2978683.0,24.130,39.72,38.12,1.650
3,f1_fuel_1994_12_2_0_7,2,1994,barry,coal,ton,3739484.0,23.950,47.21,45.99,1.970
4,f1_fuel_1994_12_2_0_10,2,1994,chickasaw,gas,mcf,40533.0,1.000,2.77,2.77,2.570


In [54]:
# total sum of null values in the fuel_data dataset
fuel_data.isnull().sum()

record_id                         0
utility_id_ferc1                  0
report_year                       0
plant_name_ferc1                  0
fuel_type_code_pudl               0
fuel_unit                       180
fuel_qty_burned                   0
fuel_mmbtu_per_unit               0
fuel_cost_per_unit_burned         0
fuel_cost_per_unit_delivered      0
fuel_cost_per_mmbtu               0
dtype: int64

In [55]:
# fuel unit is a categorical data, therefore we are using the mode of the fuel_unit to replace the missing fuel_units
mode_fuel_unit = fuel_data['fuel_unit'].mode()
mode_fuel_unit

0    mcf
dtype: object

In [56]:
# replacing the missing fuel_unit values with the mode non missing fuel_unit datasets, i.e ('mcf')
fuel_data[['fuel_unit']] = fuel_data[['fuel_unit']].fillna(value="mcf")
fuel_data.isnull().sum()

record_id                       0
utility_id_ferc1                0
report_year                     0
plant_name_ferc1                0
fuel_type_code_pudl             0
fuel_unit                       0
fuel_qty_burned                 0
fuel_mmbtu_per_unit             0
fuel_cost_per_unit_burned       0
fuel_cost_per_unit_delivered    0
fuel_cost_per_mmbtu             0
dtype: int64

In [57]:
# the fuel cost per unit burned for each fuel type
fuel_data.groupby('fuel_type_code_pudl')['fuel_cost_per_unit_burned'].mean()

fuel_type_code_pudl
coal         67.421830
gas          13.659397
nuclear    4955.157002
oil         168.877086
other        18.253856
waste        19.518122
Name: fuel_cost_per_unit_burned, dtype: float64

In [58]:
# lowest average fuel cost per unit burned
fuel_data.groupby('fuel_type_code_pudl')['fuel_cost_per_unit_burned'].mean().min()

13.659396569737135

In [59]:
# the standard deviation of the measure of energy per unit (Fuel_mmbtu_per_unit)
round(fuel_data['fuel_mmbtu_per_unit'].std(), 2)

10.6

In [60]:
# the 75th percentile of the measure of energy per unit (Fuel_mmbtu_per_unit)
round(fuel_data['fuel_mmbtu_per_unit'].quantile(q=.75),2)

17.01

In [61]:
# the skewness and kurtosis of the fuel quantity burned
fuel_qty_burned = fuel_data['fuel_qty_burned']
print("The skewness is ", skew(fuel_qty_burned))
print("The kurtosis is ", kurtosis(fuel_qty_burned))

The skewness is  15.850690077422112
The kurtosis is  651.2589365474387


In [62]:
# The percentage the missing rows
missing_feature=fuel_data['fuel_unit']
total_missing_feature = 180
total_rows = fuel_data.shape[0]
percent_missing_rows = round((total_missing_feature/total_rows) * 100, 3)
percent_missing_rows

0.61

In [63]:
# the correlation of the columns of the fuel_cost_per_unit_burned with themselves
fuel_data.corr()['fuel_cost_per_unit_burned'].sort_values()

utility_id_ferc1               -0.037863
fuel_qty_burned                -0.018535
fuel_mmbtu_per_unit            -0.010034
fuel_cost_per_mmbtu            -0.000437
fuel_cost_per_unit_delivered    0.011007
report_year                     0.013599
fuel_cost_per_unit_burned       1.000000
Name: fuel_cost_per_unit_burned, dtype: float64

In [64]:
# sorting the annual average of fuel cost per unit delivered 
fuel_data.groupby('report_year')['fuel_cost_per_unit_delivered'].mean().sort_values()

report_year
1999       25.551627
1995       32.735269
2006       38.657484
2005       41.438184
2007       43.325023
2017       46.196861
2002       47.594361
2003       55.663493
2008       58.588197
2011       59.774667
2001       60.050396
2012       60.994502
1994       63.636060
2010       91.862105
2016      103.901761
2004      139.524275
2013      172.307591
2014      192.737183
1998      287.154420
2015      326.535511
2018      499.269966
2009      652.694163
2000      985.362877
1996     9196.705948
1997    11140.197239
Name: fuel_cost_per_unit_delivered, dtype: float64

In [65]:
# filtering the dataframe to get the dataframe of coal fuel only
coal_df = fuel_data[fuel_data['fuel_type_code_pudl']=='coal']
# the percentage change in the fuel_cost_per_unit_burned in 1998 compared to 1994
change = coal_df.groupby('report_year')['fuel_cost_per_unit_burned'].mean().loc[[1994,1998]]

change.pct_change()

report_year
1994         NaN
1998   -0.124586
Name: fuel_cost_per_unit_burned, dtype: float64